In [ ]:
import pandas as pd
from COMETH import Skeleton,DynamicSkeleton
import nimblephysics as nimble
import numpy as np
import json
import time
import csv
gui = nimble.NimbleGUI()
gui.serve(8080)

# sudo lsof -iTCP:8080 -sTCP:LISTEN

In [ ]:
gui.nativeAPI().clear()
# Build skeleton
s12 = Skeleton('BODY12.xml')
# BSM
s = DynamicSkeleton(config='BODY15_constrained_3D.xml')
s.hip_correction = False
# gui.nativeAPI().renderSkeleton(s._nimble)
axis_length=0.2
gui.nativeAPI().createLine(key='x_axis',points=[np.array([0,0,0]),np.array([axis_length,0,0])],color=np.array([255,0,0,1]))
gui.nativeAPI().createLine(key='y_axis',points=[np.array([0,0,0]),np.array([0,axis_length,0])],color=np.array([0,255,0,1]))
gui.nativeAPI().createLine(key='z_axis',points=[np.array([0,0,0]),np.array([0,0,axis_length])],color=np.array([0,0,255,1]))

# Read data from CSV
# markers = pd.read_csv("~/nas/MAEVE/BEFINE/wearable_camera/TotalCapture/CommonFormat/vicon/s1/acting1/vicon_s1_acting1.csv")
# imus = pd.read_csv("~/nas/MAEVE/BEFINE/wearable_camera/TotalCapture/CommonFormat/imu/s1/acting1/imu_s1_acting1.csv")
markers = pd.read_csv("tmp/imu/vicon_s1_acting1.csv")
imus = pd.read_csv("tmp/imu/imu_s1_acting1.csv")

In [ ]:
# Build the markers dataframe with only the subset we are interested in

Rx = np.array([
    [1,0,0 ],
    [0,0,-1 ],
    [0,1,0]
])

markers.columns
markers_dict = {
'RKnee': 'right_knee',
'LWrist': 'left_wrist',
'RHip': 'right_hip',
'RShoulder': 'right_shoulder',
'LElbow': 'left_elbow',
'LHip': 'left_hip',
'RElbow': 'right_elbow',
'RWrist': 'right_wrist',
'LKnee': 'left_knee',
'LShoulder': 'left_shoulder',
'RAnkle': 'right_ankle',
'LAnkle': 'left_ankle'
 }
target = []
for i in range(markers.shape[0]):
    row = []
    for kp in markers_dict.keys():
        p = np.array([markers[markers_dict[kp]+"_x"][i],markers[markers_dict[kp]+"_y"][i],markers[markers_dict[kp]+"_z"][i]])
        # rotate the 3d point -90 on the x axis (from y up to z up)
        p_n = Rx.dot(p)
        # print(p,p_n)
        row += p_n.tolist()
    target.append(row)
target = np.array(target)

In [ ]:
# Place virtual IMUs on the shoulders
from typing import List, Tuple
right_shoulder: nimble.dynamics.BodyNode = s._nimble.getBodyNode("humerus_r")
translation: np.ndarray = np.array([0.0, 0.00, 0.0])
rotation: np.ndarray = np.eye(3)
watch_offset: nimble.math.Isometry3 = nimble.math.Isometry3(rotation, translation)
sensors: List[Tuple[nimble.dynamics.BodyNode, nimble.math.Isometry3]] = [(right_shoulder, watch_offset),]

In [ ]:
pos_imu = right_shoulder.getWorldTransform().multiply(watch_offset).matrix()[0:3,3]
s._nimble.setGravity(np.array([0.0, 0.0, -9.81]))
s._nimble.getAccelerometerReadings(sensors)

In [ ]:
# Move the body model using gt and exact qpik
kps = list(markers_dict.keys())
s.reset_history()
for i in range(target.shape[0]):
    s12.load_from_numpy(target[i,:].reshape(-1,3),s.kps)
    s.load_from_BODY12(s12)
    if i == 0:
        s.exact_scale()
    else:
        s.exact_scale(max_iterations=100)
        # s.filter([target[i,:].reshape(-1,3),])
    gui.nativeAPI().renderSkeleton(s._nimble)
    
    pos_imu = right_shoulder.getWorldTransform().multiply(watch_offset).matrix()[0:3,3]
    gui.nativeAPI().createSphere(key='imu', radii=np.array([0.02,0.02,0.02]),pos=pos_imu,color=np.array([255,255,0,0.8]))
    
    for j in range(0,int(target.shape[1]/3)):
        pos = target[i,3*j:3*j+3].reshape(3,1)
        gui.nativeAPI().createSphere(key=kps[j], radii=np.array([0.01,0.01,0.01]),pos=pos,color=np.array([0,255,0,0.8]))
    

    # Plot imu
    print(s._nimble.getAccelerometerReadings(sensors))
    
    s.reset_history()
    # time.sleep(0.1)